# Liquid-liquid equilibrium in polymer solutions

## Estimation of Flory-Huggins interaction parameter from XS test data

### Paper 3

This notebook contains an estimation procedure for the parameters of the Flory-Huggins model applied to the LLE data presented in [MATOS *et al.* (2007)](https://onlinelibrary.wiley.com/doi/full/10.1002/mren.200600003).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from plt_tools import get_figsize, set_plt_rc

from Tompa import Tompa

In [ ]:
for_paper = True
language = 'en'

plt = set_plt_rc(plt, for_paper=for_paper)

## Experimental data

In [ ]:
####################
# EXPERIMENTAL DATA
####################

z_sol = 0.97738

labels_pols = ['C2-23','C4-24','C5-27','C9-45']

teta1 = 1e-4*np.array([[1.356,1.498],[2.490,13.90],[1.286,18.50],[1.436,1.560]])

teta2 = 1e-4*np.array([[5.956,6.366],[1.261,15.11],[4.585,12.54],[7.290,5.289]])

alpha = np.array([[0.618,0.282],[0.170,0.139],[0.395,0.185],[0.458,0.148]])

mw = np.array([252.9,171.1,253.3,215])

xs = 1e-2*np.array([31.76,13.10,31.13,41.04])

# creating list of objects for each polymer
polymers = [Tompa(label=labels[i], z_sol = z_sol,
                  shulz_kind='3P', r_pol = np.arange(1e1,4e4),
                  teta1 = teta1[i], teta2 = teta2[i],
                  alpha = alpha[i],xs_fraction = xs[i]) 
            for i in range(len(labels))]


## Parameter estimation

In [ ]:
for i in range(len(polymers)):
    polymers[i].estimation()

## Plotting results

In [ ]:
if for_paper:
    fs = get_figsize(columnwidth=190, hf_abs = 160, unit='mm')

fig, ax = plt.subplots(2,2,figsize=fs,sharex=True)#,sharey=True)

for i in range(len(polymers)):
    polymers[i].plot_Experimental_Distributions(ax=ax.ravel()[i])
    polymers[i].plot_Calculated_Distributions(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(labels_pols[i]+f'\n $\chi$ = {polymers[i].A:.3f}')
    ax.ravel()[i].ticklabel_format(axis='y', style='sci', scilimits=(-4,-4))
    ax.ravel()[i].set_xlim([0,1.1e4])

handles, labels = ax[0,0].get_legend_handles_labels()
ax[-1,-1].legend(handles, labels, loc='upper right')

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Chain length, $r_i$")
plt.ylabel("Mass fraction, $w_i$")

fig.tight_layout()

plt.savefig('figures/estimation_paper3.pdf',bbox_inches='tight')

### $\chi$ as a function of XS and MW

In [ ]:
chi = [p.A for p in polymers]

if for_paper:
    fs = get_figsize(columnwidth=190, hf_abs = 80, unit='mm')

a1, b1 = np.polyfit(100*xs, chi, 1)
a2, b2 = np.polyfit(mw, chi, 1)

fig, ax = plt.subplots(1,2,figsize=fs)

ax[0].plot(100*xs,chi,'.',c='k')
ax[0].plot(100*xs,a1*100*xs+b1,c='k',label='Regression line')
ax[0].set_xlabel("XS (%)")
ax[0].set_ylabel("$\chi$")
ax[0].legend()

ax[1].plot(mw,chi,'.',c='k')
ax[1].plot(mw,a2*mw+b2,c='k',label='Regression line')
ax[1].set_xlabel("MW (g/mol)")
ax[1].set_ylabel("$\chi$")
ax[1].legend()
ax[1].ticklabel_format(axis='x', style='sci', scilimits=(4,4))


fig.tight_layout()

fig.savefig('figures/xs_dependencies_paper3.pdf',bbox_inches='tight')

Pearson correlation coefficients:

In [ ]:
np.corrcoef(xs, chi)[0,1]

In [ ]:
np.corrcoef(mw, chi)[0,1]